In [1]:
import urllib.request
import urllib.parse
import json
import re
import datetime
from collections import namedtuple
from sys import intern
import pytz

In [2]:
import sys
sys.path.append("..")
import energydata.esett

In [3]:
descriptor = {
    "start_datetime": "2022-10-29T23:00:00",
    "end_datetime": "2022-10-30T23:00:00",
    "MBA": "DK2",
    "dataset": "Prices"
}

dataset = energydata.esett.ESett(descriptor)

In [4]:
dataset.fetch_metadata()

{'fields': ['mba',
  'timestamp',
  'imbalancePurchase',
  'imbalanceSales',
  'upRegulation',
  'downRegulation',
  'valueOfAvoidedActivation',
  'incentivisingComponent',
  'mainDirection'],
 'total': 49}

In [5]:
list(dataset.fetch_data())

[{'mba': 'DK2',
  'timestamp': '2022-10-30T00:00:00',
  'imbalancePurchase': 103.01,
  'imbalanceSales': 103.01,
  'upRegulation': 103.01,
  'downRegulation': 103.01,
  'valueOfAvoidedActivation': None,
  'incentivisingComponent': None,
  'mainDirection': 0,
  'datetime_start': '2022-10-30T00:00:00+02:00',
  'datetime_end': '2022-10-30T01:00:00+02:00'},
 {'mba': 'DK2',
  'timestamp': '2022-10-30T01:00:00',
  'imbalancePurchase': 128.0,
  'imbalanceSales': 128.0,
  'upRegulation': 128.0,
  'downRegulation': 100.49,
  'valueOfAvoidedActivation': None,
  'incentivisingComponent': None,
  'mainDirection': 1,
  'datetime_start': '2022-10-30T01:00:00+02:00',
  'datetime_end': '2022-10-30T02:00:00+02:00'},
 {'mba': 'DK2',
  'timestamp': '2022-10-30T02:00:00',
  'imbalancePurchase': 128.0,
  'imbalanceSales': 128.0,
  'upRegulation': 128.0,
  'downRegulation': 99.92,
  'valueOfAvoidedActivation': None,
  'incentivisingComponent': None,
  'mainDirection': 1,
  'datetime_start': '2022-10-30T02:0

In [8]:
descriptor = {
    "start_datetime": "2022-10-29T23:00:00",
    "end_datetime": "2022-10-30T23:00:00",
    "MBA": "DK2",
    "dataset": "Prices"
}
start_dt = datetime.datetime.fromisoformat(descriptor["start_datetime"])
start_str = start_dt.astimezone(datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.000Z")
end_dt = datetime.datetime.fromisoformat(descriptor["end_datetime"])
end_str = end_dt.astimezone(datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.000Z")

query = urllib.parse.urlencode({
    "start": start_str,
    "end": end_str,
    "MBA": descriptor["MBA"]
})
url = f"https://opendata.esett.com/api/EXP14/{descriptor['dataset']}?{query}"
url

'https://opendata.esett.com/api/EXP14/Prices?start=2022-10-29T21%3A00%3A00.000Z&end=2022-10-30T22%3A00%3A00.000Z&MBA=DK2'

In [23]:
tz = pytz.timezone("Europe/Copenhagen")
dt = datetime.datetime.fromisoformat("2022-10-30T02:00:00")
tz.localize(dt, is_dst=False).isoformat()

'2022-10-30T02:00:00+01:00'

In [32]:
records = []
tz = pytz.timezone("Europe/Copenhagen")
with urllib.request.urlopen(url) as fh:
    data = json.load(fh)
    prev_timestamp = ""
    for row in data:
        record = {intern(k): v for k, v in row.items()}
        timestamp = row["timestamp"]
        is_dst = timestamp != prev_timestamp
        dt = datetime.datetime.fromisoformat(timestamp)
        start_loc = tz.localize(dt, is_dst=is_dst)
        end_loc = (start_loc.astimezone(datetime.timezone.utc) + datetime.timedelta(hours=1)).astimezone(tz)
        record[intern("datetime_start")] = start_loc.isoformat()
        record[intern("datetime_end")] = end_loc.isoformat()
        prev_timestamp = timestamp
        records.append(record)
records

[{'mba': 'DK2',
  'timestamp': '2022-10-30T00:00:00',
  'imbalancePurchase': 103.01,
  'imbalanceSales': 103.01,
  'upRegulation': 103.01,
  'downRegulation': 103.01,
  'valueOfAvoidedActivation': None,
  'incentivisingComponent': None,
  'mainDirection': 0,
  'datetime_start': '2022-10-30T00:00:00+02:00',
  'datetime_end': '2022-10-30T01:00:00+02:00'},
 {'mba': 'DK2',
  'timestamp': '2022-10-30T01:00:00',
  'imbalancePurchase': 128.0,
  'imbalanceSales': 128.0,
  'upRegulation': 128.0,
  'downRegulation': 100.49,
  'valueOfAvoidedActivation': None,
  'incentivisingComponent': None,
  'mainDirection': 1,
  'datetime_start': '2022-10-30T01:00:00+02:00',
  'datetime_end': '2022-10-30T02:00:00+02:00'},
 {'mba': 'DK2',
  'timestamp': '2022-10-30T02:00:00',
  'imbalancePurchase': 128.0,
  'imbalanceSales': 128.0,
  'upRegulation': 128.0,
  'downRegulation': 99.92,
  'valueOfAvoidedActivation': None,
  'incentivisingComponent': None,
  'mainDirection': 1,
  'datetime_start': '2022-10-30T02:0